In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset, load_data
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss, apgd_loss
from tqdm.notebook import tqdm
from architectures import load_architecture, add_lora, set_lora_gradients #load_statedict

import torch.nn as nn

args = get_args()


args.loss_function = 'APGD'

args.iterations = 20
args.pruning_ratio = 0.99
args.delta = 1

args.init_lr = 0.001

# Example Usage

# model = load_architecture(args)

# state_dict = torch.load('./state_dicts/convnext_tiny_22k_224.pth')
# model.load_state_dict(state_dict)
# model, target_layers = load_architecture(args)
# add_lora(args, model, target_layers)
# set_lora_gradients(args, model, target_layers)

# model.to('cuda')


./data


In [5]:
import timm

vit_models = timm.list_models('*deit_small_patch16_224*', pretrained=True)
for model_name in vit_models:
    print(model_name)


# vit_small_patch16_224, vit_base_patch16_224


deit_small_patch16_224.fb_in1k


In [34]:
import timm
from timm.models import create_model

# #'convnext_base_robust','convnext_base_robust',

# backbones = [ 'convnext_base',  'convnext_base.fb_in22k', 'convnext_tiny',  'convnext_tiny.fb_in22k' ]
backbones = [ 'deit_small_patch16_224.fb_in1k',  
              'vit_base_patch16_224.augreg_in1k', 
              'vit_base_patch16_224.augreg_in21k' ]

for backbone in backbones:

    model = timm.create_model(backbone, pretrained=True)
    torch.save(model.state_dict(), './state_dicts/{}.pt'.format(backbone) )



In [ ]:


class ImageNormalizer(nn.Module):
    def __init__(self, mean: Tuple[float, float, float],
        std: Tuple[float, float, float],
        persistent: bool = True) -> None:
        super(ImageNormalizer, self).__init__()

        self.register_buffer('mean', torch.as_tensor(mean).view(1, 3, 1, 1),
            persistent=persistent)
        self.register_buffer('std', torch.as_tensor(std).view(1, 3, 1, 1),
            persistent=persistent)

    def forward(self, input: Tensor) -> Tensor:
        return (input - self.mean) / self.std

def normalize_model(model: nn.Module, mean: Tuple[float, float, float],
    std: Tuple[float, float, float]) -> nn.Module:
    layers = OrderedDict([
        ('normalize', ImageNormalizer(mean, std)),
        ('model', model)
    ])
    return nn.Sequential(layers)

IMAGENET_MEAN = [c * 1. for c in (0.485, 0.456, 0.406)] #[np.array([0., 0., 0.]), np.array([0.485, 0.456, 0.406])][-1] * 255
IMAGENET_STD = [c * 1. for c in (0.229, 0.224, 0.225)] #[np.array([1., 1., 1.]), np.array([0.229, 0.224, 0.225])][-1] * 255

backbone = 'vit_base_patch16_224'

model = create_model(backbone, pretrained=False)
# model = timm.create_model(backbone, pretrained=False)
# model = normalize_model(model, IMAGENET_MEAN, IMAGENET_STD)

ckpt = torch.load('./state_dicts/weights_vit_b_50_ep.pt', map_location='cpu', weights_only=False)
ckpt = {k.replace('module.', ''): v for k, v in ckpt.items()}
ckpt = {k.replace('base_model.', ''): v for k, v in ckpt.items()}
ckpt = {k.replace('se_', 'se_module.'): v for k, v in ckpt.items()}

try:
    model.load_state_dict(ckpt)
    print('standard loading')

except:
    try:
        ckpt = {f'base_model.{k}': v for k, v in ckpt.items()}
        model.load_state_dict(ckpt)
        print('loaded from clean model')
    except:
        ckpt = {k.replace('base_model.', ''): v for k, v in ckpt.items()}
        # ckpt = {f'base_model.{k}': v for k, v in ckpt.items()}
        model.load_state_dict(ckpt)
        print('loaded')

if isinstance(model, nn.Sequential) and 'normalize' in model._modules: # remove normalization layer
    # Rebuild the sequential model without the 'normalize' layer
    model = model._modules['model']

torch.save(model.state_dict(), './state_dicts/robust_{}.pt'.format(backbone) )

standard loading


In [ ]:

ckpt = torch.load('./state_dicts/tiny_linf_wrn28-10.pt')
ckpt = {k.replace('module.0.', ''): v for k, v in ckpt['model_state_dict'].items()}
model.load_state_dict(ckpt)

torch.save(model.state_dict(), './state_dicts/robust_wideresnet_28_10.pt'.format(backbone) )

In [4]:
from architectures.wideresnetswish import wideresnet
from typing import Tuple
from torch import Tensor
import torch.nn as nn
from collections import OrderedDict
import timm
from timm.models import create_model

backbones = [ 
              'convnext_base',  'convnext_base.fb_in22k', 'robust_convnext_base',
              'convnext_tiny',  'convnext_tiny.fb_in22k', 'robust_convnext_tiny',

              'robust_wideresnet_28_10', 

              'deit_small_patch16_224.fb_in1k',
              'robust_deit_small_patch16_224',
              
              'vit_base_patch16_224.augreg_in1k',
              'vit_base_patch16_224.augreg_in21k',
              'robust_vit_base_patch16_224' ]

for backbone in backbones:
    print(backbone)
    args.backbone = backbone
    args.N = 10
    model = load_architecture(args,)
    for name, param in model.named_parameters():
        print(name)

    print()


convnext_base
stem.0.weight
stem.0.bias
stem.1.weight
stem.1.bias
stages.0.blocks.0.gamma
stages.0.blocks.0.conv_dw.weight
stages.0.blocks.0.conv_dw.bias
stages.0.blocks.0.norm.weight
stages.0.blocks.0.norm.bias
stages.0.blocks.0.mlp.fc1.weight
stages.0.blocks.0.mlp.fc1.bias
stages.0.blocks.0.mlp.fc2.weight
stages.0.blocks.0.mlp.fc2.bias
stages.0.blocks.1.gamma
stages.0.blocks.1.conv_dw.weight
stages.0.blocks.1.conv_dw.bias
stages.0.blocks.1.norm.weight
stages.0.blocks.1.norm.bias
stages.0.blocks.1.mlp.fc1.weight
stages.0.blocks.1.mlp.fc1.bias
stages.0.blocks.1.mlp.fc2.weight
stages.0.blocks.1.mlp.fc2.bias
stages.0.blocks.2.gamma
stages.0.blocks.2.conv_dw.weight
stages.0.blocks.2.conv_dw.bias
stages.0.blocks.2.norm.weight
stages.0.blocks.2.norm.bias
stages.0.blocks.2.mlp.fc1.weight
stages.0.blocks.2.mlp.fc1.bias
stages.0.blocks.2.mlp.fc2.weight
stages.0.blocks.2.mlp.fc2.bias
stages.1.downsample.0.weight
stages.1.downsample.0.bias
stages.1.downsample.1.weight
stages.1.downsample.1.bias


In [22]:
import peft

args.backbone = "robust_wideresnet_28_10"
args.N = 10
model = load_architecture(args,)
model
# Configure LoRA for intermediate layers in DeiT

# model.print_trainable_parameters()


WideResNet(
  (init_conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer): Sequential(
    (0): _BlockGroup(
      (block): Sequential(
        (0): _Block(
          (batchnorm_0): BatchNorm2d(16, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (batchnorm_1): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_1): SiLU(inplace=True)
          (conv_1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (shortcut): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): _Block(
          (batchnorm_0): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(160, 160, kernel_size=(3, 3), stride=(1,

In [3]:
from architectures.wideresnetswish import wideresnet

depth = 28
widen = 10
act_fn = 'swish'  # Assuming 'swish' is the desired activation function
num_classes = 200
model = wideresnet(depth, widen, act_fn, num_classes)
ckpt = torch.load('./state_dicts/tiny_linf_wrn28-10.pt')
ckpt = {k.replace('module.0.', ''): v for k, v in ckpt['model_state_dict'].items()}
model.load_state_dict(ckpt)

model

WideResNet(
  (init_conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer): Sequential(
    (0): _BlockGroup(
      (block): Sequential(
        (0): _Block(
          (batchnorm_0): BatchNorm2d(16, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (batchnorm_1): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_1): SiLU(inplace=True)
          (conv_1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (shortcut): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): _Block(
          (batchnorm_0): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(160, 160, kernel_size=(3, 3), stride=(1,

In [32]:
import timm

from timm.models import create_model

model = timm.create_model('convnext_tiny.fb_in22k', pretrained=True)
model_save_path = "./state_dicts/test.pth"
torch.save(model.state_dict(), model_save_path)

state_dict = torch.load('./state_dicts/test.pth')
model = timm.models.convnext.convnext_tiny(pretrained=False)
num_features = model.head.fc.in_features
model.head.fc = nn.Linear(num_features, 21841)  
model.load_state_dict(state_dict)

In [19]:
import timm
import torch

equivalencies = { 'convnext_base':'convnext_base',
                      'convnext_base.fb_in22k':'convnext_base.fb_in22k', 
                      'robust_convnext_base':'convnext_base',
                      
                      'convnext_tiny':'convnext_tiny',
                      'convnext_tiny.fb_in22k':'convnext_tiny.fb_in22k',
                      'robust_convnext_tiny':'convnext_tiny',

                      'robust_wideresnet_28_10': 'robust_wideresnet_28_10',

                      'deit_small_patch16_224.fb_in1k': 'deit_small_patch16_224.fb_in1k',
                      'robust_deit_small_patch16_224': 'deit_small_patch16_224',

                      'vit_base_patch16_224.augreg_in1k':'vit_base_patch16_224.augreg_in1k',
                      'vit_base_patch16_224.augreg_in21k':'vit_base_patch16_224.augreg_in21k',
                      'robust_vit_base_patch16_224': 'vit_base_patch16_224'
                           
                }

backbone = 'robust_wideresnet_28_10'

from architectures.wideresnetswish import wideresnet
model = wideresnet(depth = 28, widen = 10, act_fn = 'swish', num_classes = 200)

# model = timm.create_model( equivalencies[backbone], pretrained=False )
state_dict = torch.load('./state_dicts/{}.pt'.format(backbone) )
model.load_state_dict(state_dict)

<All keys matched successfully>

In [20]:
model

WideResNet(
  (init_conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer): Sequential(
    (0): _BlockGroup(
      (block): Sequential(
        (0): _Block(
          (batchnorm_0): BatchNorm2d(16, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (batchnorm_1): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_1): SiLU(inplace=True)
          (conv_1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (shortcut): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): _Block(
          (batchnorm_0): BatchNorm2d(160, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
          (relu_0): SiLU(inplace=True)
          (conv_0): Conv2d(160, 160, kernel_size=(3, 3), stride=(1,

In [5]:
num_features = model.head.in_features
model.head = nn.Linear(num_features, 20)

In [2]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(